In [ ]:
!pip install tensorflow

In [ ]:
# imports

In [ ]:
# data manipulation and visualization libraries\nimport os\nimport cv2\nimport numpy as np\nimport pandas as pd\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nfrom sklearn.metrics import confusion_matrix\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import LabelEncoder

In [ ]:
# cnn supporting libraries\nfrom keras.models import Sequential\nfrom keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout\nfrom keras.utils import to_categorical\nfrom keras.models import load_model\nfrom keras.callbacks import ModelCheckpoint\nfrom tensorflow.keras import backend as K\nfrom tensorflow.keras.optimizers import Adam

In [ ]:
# resnet supporting libraries\nimport tensorflow as tf\nfrom tensorflow.keras.applications.resnet50 import ResNet50\nfrom tensorflow.keras.preprocessing import image\nfrom tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions\nfrom tensorflow.keras.utils import to_categorical

In [ ]:
!pip freeze > requirements.txt

In [ ]:
# preprocess the images

In [ ]:
def preprocess_data(image_path):\n    # load image using cv2\n    image_bgr = cv2.imread(image_path)\n    # convert image from bgr to rgb\n    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)\n    # resize image to make each same size\n    resized_image = cv2.resize(image_rgb, (64, 64))\n    return resized_image

In [ ]:
# defining directory where all images are stored\nmain_directory = 'bone_marrow_cell_dataset'\n\n# list of 21 types of cells\n

In [ ]:
i = 1\n# reading and preprocessing images\nfor label in labels:\n    folder_path = os.path.join(main_directory, label)   # folder path \n    if os.path.exists(folder_path):\n        for filename in os.listdir(folder_path):\n            file_path = os.path.join(folder_path, filename)  # file path\n            if os.path.isdir(file_path): \n                for img_filename in os.listdir(file_path):\n                    if img_filename.endswith('.jpg'):\n                        row_img_path = os.path.join(file_path, img_filename)  # image path\n                        img_path = .join(row_img_path.split())\n                        images_list.append(preprocess_data(img_path))\n                        cell_types_list.append(label)\n                        print(i, , img_path)\n                        i+=1

In [ ]:
# converting the images and labels to NumPy arrays\nimages_array = np.array(images_list)\ncell_types_array = np.array(cell_types_list)\n\nprint(,images_array.shape)\nprint(,cell_types_array.shape)\n\nprint(

In [ ]:
# display image of unique cell\ntotal = [preprocess_data('bone_marrow_cell_dataset/EBO/0001-1000/EBO_00001.jpg'),\n         preprocess_data('bone_marrow_cell_dataset/EOS/0001-1000/EOS_00001.jpg'),\n         preprocess_data('bone_marrow_cell_dataset/LYT/0001-1000/LYT_00001.jpg'),\n         preprocess_data('bone_marrow_cell_dataset/MON/0001-1000/MON_00001.jpg'),\n         preprocess_data('bone_marrow_cell_dataset/NGB/0001-1000/NGB_00001.jpg'),\n

In [ ]:
# perform one-hot encoding on cell types\nlabel_encoder = LabelEncoder()\nlabels_encoded = label_encoder.fit_transform(cell_types_array)\ncell_type = to_categorical(labels_encoded, num_classes=21)

In [ ]:
# split the data into training and testing sets with ratio 80:20\nX_train, X_test, y_train, y_test = train_test_split(images_array, cell_type, test_size=0.2, random_state=42)

In [ ]:
# Approach 1: CNN model

In [ ]:
# define cnn model using keras sequantial api\nmodel = Sequential()\nmodel.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=(64, 64, 3))) \nmodel.add(MaxPooling2D(pool_size = (2, 2)))\n\nmodel.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))\nmodel.add(MaxPooling2D(pool_size = (2, 2)))\n\nmodel.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu')) \nmodel.add(MaxPooling2D(pool_size = (2, 2)))\n\nmodel.add(Flatten())\nmodel.add(Dense (128, activation = 'relu'))\nmodel.add(Dropout (0.20))\n\nmodel.add(Dense (21, activation = 'softmax'))  # 21: cell types

In [ ]:
# summary of model\nmodel.summary()

In [ ]:
# compile model for multiclass classification\n

In [ ]:
# define the path to save the best model\nmodel_checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
# train model for 10 epochs\n

In [ ]:
model.save()

In [ ]:
# visuals of accuracy throughout epochs\n

In [ ]:
# evaluate the model on the testing set\nloss, accuracy = model.evaluate(X_test, y_test)\nprint(f'Loss during: {loss:.4f}')\nprint(f'Model accuracy: {accuracy:.4f}')

In [ ]:
# load the saved model while making checkoint\nmodel_path = 'model.h5'\nloaded_model = load_model(model_path)\n\n# perform predictions on test set\npredicted = loaded_model.predict(X_test)\n\n# convert predictions to class labels\npredicted_cnn_cells = label_encoder.inverse_transform(np.argmax(predicted, axis=1))\n\n# convert one-hot encoded true labels to class labels\ntrue_cnn_cells = label_encoder.inverse_transform(np.argmax(y_test, axis=1))

In [ ]:
# plotting predicted results\nplt.figure(figsize=(10, 10))\nfor i in range(9):\n    plt.subplot(3, 3, i+1)\n

In [ ]:
confusion = confusion_matrix(true_cnn_cells, predicted_cnn_cells)\n\n# plot the confusion matrix\nplt.figure(figsize=(8, 6))\nsns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)\nplt.title('Confusion Matrix')\nplt.xlabel('Predicted')\nplt.ylabel('True')\nplt.show()

In [ ]:
# define class vise accuracy\nclass_accuracy = confusion.diagonal() / confusion.sum(axis=1)\n

In [ ]:
# Approach 2: ResNet

In [ ]:
X = images_array # features(image array)\ny = cell_type # target(cell type)

In [ ]:
# split data into training and testing by 80:20\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# calling pre-rained 50-layer ResNet \nbase_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [ ]:
# build this dataset's training model on resnet\nmodel = tf.keras.models.Sequential([\n    base_model,\n    tf.keras.layers.GlobalAveragePooling2D(),\n    tf.keras.layers.Dense(128, activation='relu'),\n    tf.keras.layers.Dropout(0.5),\n    tf.keras.layers.Dense(21, activation='softmax')\n

In [ ]:
# train model\nresnet_model = model.fit(X_train, y_train, epochs=5, batch_size=300, validation_data=(X_test, y_test))

In [ ]:
model.save()  # save model

In [ ]:
# visuals of accuracy throughout epochs\n

In [ ]:
# evaluate the model on the testing set\nloss, accuracy = model.evaluate(X_test, y_test)\nprint(f'Loss during: {loss:.4f}')\nprint(f'Model accuracy: {accuracy:.4f}')

In [ ]:
# load the saved model while making checkoint\nmodel_path = 'bone_marrow_resnet_model.h5'\nloaded_model = load_model(model_path)\n\n# perform predictions on test set\npredicted = loaded_model.predict(X_test)\n\n# convert predictions to class labels\npredicted_resnet_cells = label_encoder.inverse_transform(np.argmax(predicted, axis=1))\n\n# convert one-hot encoded true labels to class labels\ntrue_resnet_cells = label_encoder.inverse_transform(np.argmax(y_test, axis=1))

In [ ]:
# plotting predicted results\nplt.figure(figsize=(10, 10))\nfor i in range(9):\n    plt.subplot(3, 3, i+1)\n

In [ ]:
confusion = confusion_matrix(true_resnet_cells, predicted_resnet_cells)\n\n# plot confusion matrix\nplt.figure(figsize=(8, 6))\nsns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)\nplt.title('Confusion Matrix')\nplt.xlabel('Predicted')\nplt.ylabel('True')\nplt.show()

In [ ]:
# define class vise accuracy\nclass_accuracy = confusion.diagonal() / confusion.sum(axis=1)\n

In [ ]:
# comparison between two models

In [ ]:
# accuracy comparison\nplt.figure(figsize=(10,5))\n

In [ ]:
# validation accuracy comparison\nplt.figure(figsize=(20,5))\n